In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
# =========================
# 1️⃣ Install Libraries
# =========================
!pip install torch torchvision opencv-python einops tqdm

# =========================
# 2️⃣ Imports
# =========================
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from einops import rearrange
from tqdm import tqdm

# =========================
# 3️⃣ Dataset Paths (Windows)
# =========================
BASE_PATH = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw"

REAL_PATH = os.path.join(BASE_PATH, "real")
FAKE_PATH = os.path.join(BASE_PATH, "fake")

DATASET = {
    "Real": REAL_PATH,
    "Fake": FAKE_PATH
}

LABELS = {
    "Real": 0,
    "Fake": 1
}

# =========================
# 4️⃣ Dataset Class
# =========================
class DeepFakeDataset(Dataset):
    def __init__(self, paths, labels, frames=16):
        self.samples = []
        self.frames = frames

        for k, p in paths.items():
            for vid in os.listdir(p):
                if vid.endswith(".mp4"):
                    self.samples.append((os.path.join(p, vid), labels[k]))

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def extract_frames(self, video):
        cap = cv2.VideoCapture(video)
        if not cap.isOpened():
            print("❌ Impossible d'ouvrir:", video)
            return None

        frames = []
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total == 0:
            print("❌ Vidéo vide:", video)
            cap.release()
            return None

        idxs = torch.linspace(0, total - 1, self.frames).long().tolist()
        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.transform(frame)
                frames.append(frame)
        cap.release()

        if len(frames) == 0:
            return None
        return torch.stack(frames)

    def __getitem__(self, idx):
        while True:
            video, label = self.samples[idx]
            frames = self.extract_frames(video)
            if frames is not None:
                return frames, torch.tensor(label)
            idx = (idx + 1) % len(self.samples)

    def __len__(self):
        return len(self.samples)

# =========================
# 5️⃣ Neural Oscillator Layer
# =========================
class NeuralOscillator(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.freq = nn.Parameter(torch.randn(dim))
        self.phase = nn.Parameter(torch.randn(dim))

    def forward(self, x):
        t = torch.arange(x.size(1), device=x.device).float()
        osc = torch.sin(self.freq * t.unsqueeze(-1) + self.phase)
        return x * osc

# =========================
# 6️⃣ Quantum Attention Layer
# =========================
class QuantumAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.phase = nn.Linear(dim, dim)

    def forward(self, x):
        phase = self.phase(x)
        amp = torch.abs(torch.sum(x * torch.exp(1j * phase), dim=1))
        return amp.real

# =========================
# 7️⃣ Full Model: QINON
# =========================
class QINON(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()  # 512 features per frame

        self.osc = NeuralOscillator(512)
        self.attn = QuantumAttention(512)
        self.fc = nn.Linear(512, 1)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = rearrange(x, 'b t c h w -> (b t) c h w')
        feats = self.backbone(x)
        feats = rearrange(feats, '(b t) d -> b t d', b=b)
        feats = self.osc(feats)
        feats = self.attn(feats)
        out = self.fc(feats)
        return torch.sigmoid(out)

# =========================
# 8️⃣ Dataset + DataLoader
# =========================
dataset = DeepFakeDataset(DATASET, LABELS, frames=16)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

# =========================
# 9️⃣ Initialize Model
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = QINON().to(device)

# =========================
# 🔟 Optimizer + Loss
# =========================
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

# =========================
# 1️⃣1️⃣ Training Loop
# =========================
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device).float().unsqueeze(1)
        preds = model(x)
        loss = criterion(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(loader):.4f}")

# =========================
# 1️⃣2️⃣ Test single batch
# =========================
model.eval()
with torch.no_grad():
    x, y = next(iter(loader))
    x, y = x.to(device), y.to(device).float().unsqueeze(1)
    out = model(x)
    print("Output:", out)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\EliteLaptop/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.0MB/s]
100%|██████████| 953/953 [11:49<00:00,  1.34it/s]


Epoch [1/5] Loss: 0.5877


100%|██████████| 953/953 [11:58<00:00,  1.33it/s]


Epoch [2/5] Loss: 0.4840


100%|██████████| 953/953 [11:28<00:00,  1.38it/s]


Epoch [3/5] Loss: 0.4756


100%|██████████| 953/953 [10:23<00:00,  1.53it/s]


Epoch [4/5] Loss: 0.4735


100%|██████████| 953/953 [10:34<00:00,  1.50it/s]


Epoch [5/5] Loss: 0.4707
Output: tensor([[0.8440]], device='cuda:0')


In [3]:
# =========================
# 1️⃣ Install Libraries
# =========================
!pip install torch torchvision opencv-python einops tqdm scikit-learn

# =========================
# 2️⃣ Imports
# =========================
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from einops import rearrange
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score

# =========================
# 3️⃣ Dataset Paths (Windows)
# =========================
BASE_PATH = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw"

REAL_PATH = os.path.join(BASE_PATH, "real")
FAKE_PATH = os.path.join(BASE_PATH, "fake")

DATASET = {
    "Real": REAL_PATH,
    "Fake": FAKE_PATH
}

LABELS = {
    "Real": 0,
    "Fake": 1
}

# =========================
# 4️⃣ Dataset Class
# =========================
class DeepFakeDataset(Dataset):
    def __init__(self, paths, labels, frames=50):
        self.samples = []
        self.frames = frames

        for k, p in paths.items():
            for vid in os.listdir(p):
                if vid.endswith(".mp4"):
                    self.samples.append((os.path.join(p, vid), labels[k]))

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def extract_frames(self, video):
        cap = cv2.VideoCapture(video)
        if not cap.isOpened():
            print("❌ Impossible d'ouvrir:", video)
            return None

        frames = []
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total == 0:
            print("❌ Vidéo vide:", video)
            cap.release()
            return None

        idxs = torch.linspace(0, total - 1, self.frames).long().tolist()
        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.transform(frame)
                frames.append(frame)
        cap.release()

        if len(frames) == 0:
            return None
        return torch.stack(frames)

    def __getitem__(self, idx):
        while True:
            video, label = self.samples[idx]
            frames = self.extract_frames(video)
            if frames is not None:
                return frames, torch.tensor(label)
            idx = (idx + 1) % len(self.samples)

    def __len__(self):
        return len(self.samples)

# =========================
# 5️⃣ Neural Oscillator Layer
# =========================
class NeuralOscillator(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.freq = nn.Parameter(torch.randn(dim))
        self.phase = nn.Parameter(torch.randn(dim))

    def forward(self, x):
        t = torch.arange(x.size(1), device=x.device).float()
        osc = torch.sin(self.freq * t.unsqueeze(-1) + self.phase)
        return x * osc

# =========================
# 6️⃣ Quantum Attention Layer
# =========================
class QuantumAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.phase = nn.Linear(dim, dim)

    def forward(self, x):
        phase = self.phase(x)
        amp = torch.abs(torch.sum(x * torch.exp(1j * phase), dim=1))
        return amp.real

# =========================
# 7️⃣ Full Model: QINON
# =========================
class QINON(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()  # 512 features per frame

        self.osc = NeuralOscillator(512)
        self.attn = QuantumAttention(512)
        self.fc = nn.Linear(512, 1)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = rearrange(x, 'b t c h w -> (b t) c h w')
        feats = self.backbone(x)
        feats = rearrange(feats, '(b t) d -> b t d', b=b)
        feats = self.osc(feats)
        feats = self.attn(feats)
        out = self.fc(feats)
        return torch.sigmoid(out)

# =========================
# 8️⃣ Dataset + DataLoader
# =========================
dataset = DeepFakeDataset(DATASET, LABELS, frames=100)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

# =========================
# 9️⃣ Initialize Model
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = QINON().to(device)

# =========================
# 🔟 Optimizer + Loss
# =========================
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

# =========================
# 1️⃣1️⃣ Training Loop avec Accuracy & AUC
# =========================
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device).float().unsqueeze(1)
        preds = model(x)
        loss = criterion(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        all_labels.extend(y.cpu().numpy())
        all_preds.extend(preds.detach().cpu().numpy())

    binary_preds = [1 if p >= 0.5 else 0 for p in all_preds]
    acc = accuracy_score(all_labels, binary_preds)
    try:
        auc = roc_auc_score(all_labels, all_preds)
    except:
        auc = 0.0

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(loader):.4f} | Accuracy: {acc:.4f} | AUC: {auc:.4f}")

# =========================
# 1️⃣2️⃣ Test single batch avec Accuracy & AUC
# =========================
model.eval()
with torch.no_grad():
    x, y = next(iter(loader))
    x, y = x.to(device), y.to(device).float().unsqueeze(1)
    out = model(x)

    binary_out = [1 if p >= 0.5 else 0 for p in out.cpu().numpy()]
    acc = accuracy_score(y.cpu().numpy(), binary_out)
    try:
        auc = roc_auc_score(y.cpu().numpy(), out.cpu().numpy())
    except:
        auc = 0.0

    print("Output probabilities:", out.cpu().numpy())
    print(f"Test Accuracy: {acc:.4f} | Test AUC: {auc:.4f}")


100%|██████████| 953/953 [27:05<00:00,  1.71s/it]


Epoch [1/5] Loss: 1.2616 | Accuracy: 0.7335 | AUC: 0.4641


100%|██████████| 953/953 [25:50<00:00,  1.63s/it]


Epoch [2/5] Loss: 0.6234 | Accuracy: 0.7838 | AUC: 0.4930


100%|██████████| 953/953 [26:15<00:00,  1.65s/it]


...

100%|██████████| 953/953 [25:44<00:00,  1.62s/it]


Epoch [4/5] Loss: 0.4982 | Accuracy: 0.8195 | AUC: 0.5610


100%|██████████| 953/953 [24:38<00:00,  1.55s/it]


Epoch [5/5] Loss: 0.5074 | Accuracy: 0.8195 | AUC: 0.5583
Output probabilities: [[0.6556499]]
Test Accuracy: 1.0000 | Test AUC: nan


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [6]:
# =========================================================
# Quantum-Inspired Neural Oscillator Network (QINON)
# DeepFake Detection – Stable & Improved Version
# =========================================================

# =========================
# 1️⃣ Imports
# =========================
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from einops import rearrange
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# =========================
# 2️⃣ Paths (Windows)
# =========================
BASE_PATH = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw"

DATASET_PATHS = {
    "Real": os.path.join(BASE_PATH, "real"),
    "Fake": os.path.join(BASE_PATH, "fake")
}

LABELS = {"Real": 0, "Fake": 1}

# =========================
# 3️⃣ Dataset
# =========================
class DeepFakeDataset(Dataset):
    def __init__(self, samples, frames=32):
        self.samples = samples
        self.frames = frames
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def extract_frames(self, video):
        cap = cv2.VideoCapture(video)
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total <= 0:
            return None

        idxs = torch.linspace(0, total - 1, self.frames).long().tolist()
        frames = []

        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.transform(frame)
                frames.append(frame)

        cap.release()
        return torch.stack(frames) if len(frames) > 0 else None

    def __getitem__(self, idx):
        while True:
            path, label = self.samples[idx]
            frames = self.extract_frames(path)
            if frames is not None:
                return frames, torch.tensor(label, dtype=torch.float32)
            idx = (idx + 1) % len(self.samples)

    def __len__(self):
        return len(self.samples)

# =========================
# 4️⃣ Neural Oscillator (Stable)
# =========================
class NeuralOscillator(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.freq = nn.Parameter(torch.randn(dim) * 0.1)
        self.phase = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        t = torch.linspace(0, 1, x.size(1), device=x.device)
        osc = torch.sin(2 * torch.pi * self.freq * t.unsqueeze(-1) + self.phase)
        return x * osc

# =========================
# 5️⃣ Quantum-Inspired Attention (Real & Stable)
# =========================
class QuantumAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.phase = nn.Linear(dim, dim)
        self.amplitude = nn.Linear(dim, 1)

    def forward(self, x):
        phase = torch.tanh(self.phase(x))
        amp = torch.softmax(self.amplitude(x), dim=1)
        return torch.sum(x * amp * phase, dim=1)

# =========================
# 6️⃣ QINON Model
# =========================
class QINON(nn.Module):
    def __init__(self):
        super().__init__()

        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()

        for p in self.backbone.parameters():
            p.requires_grad = False
        for p in self.backbone.layer4.parameters():
            p.requires_grad = True

        self.osc = NeuralOscillator(512)
        self.attn = QuantumAttention(512)
        self.fc = nn.Linear(512, 1)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = rearrange(x, 'b t c h w -> (b t) c h w')
        feats = self.backbone(x)
        feats = rearrange(feats, '(b t) d -> b t d', b=b)
        feats = self.osc(feats)
        feats = self.attn(feats)
        return self.fc(feats)  # logits

# =========================
# 7️⃣ Prepare Data
# =========================
samples = []
for cls, path in DATASET_PATHS.items():
    for v in os.listdir(path):
        if v.endswith(".mp4"):
            samples.append((os.path.join(path, v), LABELS[cls]))

train_s, val_s = train_test_split(
    samples, test_size=0.2, stratify=[l for _, l in samples]
)

train_ds = DeepFakeDataset(train_s)
val_ds = DeepFakeDataset(val_s)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

# =========================
# 8️⃣ Training Setup
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = QINON().to(device)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

# =========================
# 9️⃣ Training Loop
# =========================
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(device), y.to(device).unsqueeze(1)
        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ---------- Validation ----------
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            logits = model(x)
            probs = torch.sigmoid(logits)
            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(y.numpy())

    acc = accuracy_score(all_labels, [p >= 0.5 for p in all_preds])
    auc = roc_auc_score(all_labels, all_preds)

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Loss: {train_loss/len(train_loader):.4f} "
          f"| Val Acc: {acc:.4f} | Val AUC: {auc:.4f}")



Epoch 1: 100%|██████████| 1167/1167 [34:39<00:00,  1.78s/it] 


Epoch [1/5] Loss: 0.6416 | Val Acc: 0.4897 | Val AUC: 0.5943


Epoch 2: 100%|██████████| 1167/1167 [33:26<00:00,  1.72s/it] 


Epoch [2/5] Loss: 0.4397 | Val Acc: 0.5479 | Val AUC: 0.6074


Epoch 3: 100%|██████████| 1167/1167 [31:10<00:00,  1.60s/it] 


Epoch [3/5] Loss: 0.3318 | Val Acc: 0.5479 | Val AUC: 0.6713


Epoch 4: 100%|██████████| 1167/1167 [26:58<00:00,  1.39s/it]


Epoch [4/5] Loss: 0.2993 | Val Acc: 0.6233 | Val AUC: 0.6732


Epoch 5: 100%|██████████| 1167/1167 [31:10<00:00,  1.60s/it] 


Epoch [5/5] Loss: 0.2631 | Val Acc: 0.5582 | Val AUC: 0.5885


In [7]:
# =========================================================
# QINON + Temporal FFT
# Quantum-Inspired Neural Oscillator Network
# DeepFake Detection (Improved AUC Version)
# =========================================================

# =========================
# 1️⃣ Imports
# =========================
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from einops import rearrange
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# =========================
# 2️⃣ Paths (Windows)
# =========================
BASE_PATH = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw"

DATASET_PATHS = {
    "Real": os.path.join(BASE_PATH, "real"),
    "Fake": os.path.join(BASE_PATH, "fake")
}

LABELS = {"Real": 0, "Fake": 1}

# =========================
# 3️⃣ Dataset
# =========================
class DeepFakeDataset(Dataset):
    def __init__(self, samples, frames=32):
        self.samples = samples
        self.frames = frames
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def extract_frames(self, video):
        cap = cv2.VideoCapture(video)
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total <= 0:
            cap.release()
            return None

        idxs = torch.linspace(0, total - 1, self.frames).long().tolist()
        frames = []

        for i in range(total):
            ret, frame = cap.read()
            if not ret:
                break
            if i in idxs:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.transform(frame)
                frames.append(frame)

        cap.release()
        return torch.stack(frames) if len(frames) > 0 else None

    def __getitem__(self, idx):
        while True:
            path, label = self.samples[idx]
            frames = self.extract_frames(path)
            if frames is not None:
                return frames, torch.tensor(label, dtype=torch.float32)
            idx = (idx + 1) % len(self.samples)

    def __len__(self):
        return len(self.samples)

# =========================
# 4️⃣ Neural Oscillator
# =========================
class NeuralOscillator(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.freq = nn.Parameter(torch.randn(dim) * 0.1)
        self.phase = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        # x: [B, T, D]
        t = torch.linspace(0, 1, x.size(1), device=x.device)
        osc = torch.sin(2 * torch.pi * self.freq * t.unsqueeze(-1) + self.phase)
        return x * osc

# =========================
# 5️⃣ Quantum-Inspired Attention
# =========================
class QuantumAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.phase = nn.Linear(dim, dim)
        self.amplitude = nn.Linear(dim, 1)

    def forward(self, x):
        phase = torch.tanh(self.phase(x))
        amp = torch.softmax(self.amplitude(x), dim=1)
        return torch.sum(x * amp * phase, dim=1)

# =========================
# 6️⃣ Temporal FFT Branch
# =========================
class TemporalFFT(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim, dim)

    def forward(self, x):
        # x: [B, T, D]
        fft = torch.fft.rfft(x, dim=1)
        mag = torch.abs(fft).mean(dim=1)
        return self.fc(mag)

# =========================
# 7️⃣ QINON + FFT Model
# =========================
class QINON(nn.Module):
    def __init__(self):
        super().__init__()

        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()

        for p in self.backbone.parameters():
            p.requires_grad = False
        for p in self.backbone.layer4.parameters():
            p.requires_grad = True

        self.osc = NeuralOscillator(512)
        self.attn = QuantumAttention(512)
        self.fft = TemporalFFT(512)

        self.fc = nn.Linear(1024, 1)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = rearrange(x, 'b t c h w -> (b t) c h w')
        feats = self.backbone(x)
        feats = rearrange(feats, '(b t) d -> b t d', b=b)

        osc_feats = self.osc(feats)
        attn_feats = self.attn(osc_feats)   # [B, 512]
        fft_feats = self.fft(feats)         # [B, 512]

        fused = torch.cat([attn_feats, fft_feats], dim=1)
        return self.fc(fused)  # logits

# =========================
# 8️⃣ Prepare Data
# =========================
samples = []
for cls, path in DATASET_PATHS.items():
    for v in os.listdir(path):
        if v.endswith(".mp4"):
            samples.append((os.path.join(path, v), LABELS[cls]))

train_s, val_s = train_test_split(
    samples,
    test_size=0.2,
    stratify=[l for _, l in samples]
)

train_ds = DeepFakeDataset(train_s)
val_ds = DeepFakeDataset(val_s)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

# =========================
# 9️⃣ Training Setup
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = QINON().to(device)

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)

criterion = nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=1
)

# =========================
# 🔟 Training Loop
# =========================
EPOCHS = 8
best_auc = 0.0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x = x.to(device)
        y = y.to(device).unsqueeze(1)

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # ---------- Validation ----------
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            logits = model(x)
            probs = torch.sigmoid(logits)

            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(y.numpy())

    acc = accuracy_score(all_labels, [p >= 0.5 for p in all_preds])
    auc = roc_auc_score(all_labels, all_preds)

    scheduler.step(auc)

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Loss: {train_loss/len(train_loader):.4f} "
          f"| Val Acc: {acc:.4f} | Val AUC: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), "best_qinon_fft.pth")

print(f"\n✅ Best Validation AUC: {best_auc:.4f}")


Epoch 1: 100%|██████████| 1167/1167 [29:58<00:00,  1.54s/it] 


Epoch [1/8] Loss: 0.5995 | Val Acc: 0.3459 | Val AUC: 0.1312


Epoch 2: 100%|██████████| 1167/1167 [29:06<00:00,  1.50s/it] 


Epoch [2/8] Loss: 0.3722 | Val Acc: 0.2260 | Val AUC: 0.1630


Epoch 3: 100%|██████████| 1167/1167 [35:11<00:00,  1.81s/it] 


...

Epoch [4/8] Loss: 0.2789 | Val Acc: 0.3185 | Val AUC: 0.1606


Epoch 5: 100%|██████████| 1167/1167 [32:00<00:00,  1.65s/it] 


Epoch [5/8] Loss: 0.2438 | Val Acc: 0.2808 | Val AUC: 0.2091


Epoch 6:  69%|██████▉   | 811/1167 [1:37:16<42:41,  7.20s/it]      


KeyboardInterrupt: 